In [7]:
import requests
import pandas as pd
import numpy as np
import math
from pandas import json_normalize 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
# season stats per team
response = requests.get('https://api.sportsdata.io/v3/nfl/scores/json/TeamSeasonStats/2020REG?key=d8b5ea01537141eb9a320f95994b7109')

# put in dataframe
team_stats_2020 = pd.DataFrame.from_dict(response.json())

# calculate average score
team_stats_2020['average_score'] = round((team_stats_2020['Score'] / team_stats_2020['Games']), 2)

# display first few
team_stats_2020.head()

# this one has a lot of data, it is condensed in this view (for whole thing check here: https://api.sportsdata.io/v3/nfl/scores/json/TeamSeasonStats/2020REG?key=d8b5ea01537141eb9a320f95994b7109)

,SeasonType,Season,Team,Score,OpponentScore,TotalScore,Temperature,Humidity,WindSpeed,OverUnder,PointSpread,ScoreQuarter1,ScoreQuarter2,ScoreQuarter3,ScoreQuarter4,ScoreOvertime,TimeOfPossession,FirstDowns,FirstDownsByRushing,FirstDownsByPassing,FirstDownsByPenalty,OffensivePlays,OffensiveYards,OffensiveYardsPerPlay,Touchdowns,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,PassingAttempts,PassingCompletions,PassingYards,PassingTouchdowns,PassingInterceptions,PassingYardsPerAttempt,PassingYardsPerCompletion,CompletionPercentage,PasserRating,ThirdDownAttempts,ThirdDownConversions,ThirdDownPercentage,FourthDownAttempts,FourthDownConversions,FourthDownPercentage,RedZoneAttempts,RedZoneConversions,GoalToGoAttempts,GoalToGoConversions,ReturnYards,Penalties,PenaltyYards,Fumbles,FumblesLost,TimesSacked,TimesSackedYards,QuarterbackHits,TacklesForLoss,Safeties,Punts,PuntYards,PuntAverage,Giveaways,Takeaways,TurnoverDifferential,OpponentScoreQuarter1,OpponentScoreQuarter2,OpponentScoreQuarter3,OpponentScoreQuarter4,OpponentScoreOvertime,OpponentTimeOfPossession,OpponentFirstDowns,OpponentFirstDownsByRushing,OpponentFirstDownsByPassing,OpponentFirstDownsByPenalty,OpponentOffensivePlays,OpponentOffensiveYards,OpponentOffensiveYardsPerPlay,OpponentTouchdowns,OpponentRushingAttempts,OpponentRushingYards,OpponentRushingYardsPerAttempt,OpponentRushingTouchdowns,OpponentPassingAttempts,OpponentPassingCompletions,OpponentPassingYards,OpponentPassingTouchdowns,OpponentPassingInterceptions,OpponentPassingYardsPerAttempt,OpponentPassingYardsPerCompletion,OpponentCompletionPercentage,OpponentPasserRating,OpponentThirdDownAttempts,OpponentThirdDownConversions,OpponentThirdDownPercentage,OpponentFourthDownAttempts,OpponentFourthDownConversions,OpponentFourthDownPercentage,OpponentRedZoneAttempts,OpponentRedZoneConversions,OpponentGoalToGoAttempts,OpponentGoalToGoConversions,OpponentReturnYards,OpponentPenalties,OpponentPenaltyYards,OpponentFumbles,OpponentFumblesLost,OpponentTimesSacked,OpponentTimesSackedYards,OpponentQuarterbackHits,OpponentTacklesForLoss,OpponentSafeties,OpponentPunts,OpponentPuntYards,OpponentPuntAverage,OpponentGiveaways,OpponentTakeaways,OpponentTurnoverDifferential,RedZonePercentage,GoalToGoPercentage,QuarterbackHitsDifferential,TacklesForLossDifferential,QuarterbackSacksDifferential,TacklesForLossPercentage,QuarterbackHitsPercentage,TimesSackedPercentage,OpponentRedZonePercentage,OpponentGoalToGoPercentage,OpponentQuarterbackHitsDifferential,OpponentTacklesForLossDifferential,OpponentQuarterbackSacksDifferential,OpponentTacklesForLossPercentage,OpponentQuarterbackHitsPercentage,OpponentTimesSackedPercentage,Kickoffs,KickoffsInEndZone,KickoffTouchbacks,PuntsHadBlocked,PuntNetAverage,ExtraPointKickingAttempts,ExtraPointKickingConversions,ExtraPointsHadBlocked,ExtraPointPassingAttempts,ExtraPointPassingConversions,ExtraPointRushingAttempts,ExtraPointRushingConversions,FieldGoalAttempts,FieldGoalsMade,FieldGoalsHadBlocked,PuntReturns,PuntReturnYards,KickReturns,KickReturnYards,InterceptionReturns,InterceptionReturnYards,OpponentKickoffs,OpponentKickoffsInEndZone,OpponentKickoffTouchbacks,OpponentPuntsHadBlocked,OpponentPuntNetAverage,OpponentExtraPointKickingAttempts,OpponentExtraPointKickingConversions,OpponentExtraPointsHadBlocked,OpponentExtraPointPassingAttempts,OpponentExtraPointPassingConversions,OpponentExtraPointRushingAttempts,OpponentExtraPointRushingConversions,OpponentFieldGoalAttempts,OpponentFieldGoalsMade,OpponentFieldGoalsHadBlocked,OpponentPuntReturns,OpponentPuntReturnYards,OpponentKickReturns,OpponentKickReturnYards,OpponentInterceptionReturns,OpponentInterceptionReturnYards,SoloTackles,AssistedTackles,Sacks,SackYards,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,InterceptionReturnTouchdowns,BlockedKicks,PuntReturnTouchdowns,PuntReturnLong,KickReturnTouchdowns,KickReturnLong,BlockedKickReturnYards,BlockedKickReturnTouchdowns,FieldGoalReturnYards,Fie

In [9]:
# season standing per team
response = requests.get('https://api.sportsdata.io/v3/nfl/scores/json/Standings/2020REG?key=d8b5ea01537141eb9a320f95994b7109')

# put in dataframe
team_standings_2020 = pd.DataFrame.from_dict(response.json())

# display first few
team_standings_2020.head()

,SeasonType,Season,Conference,Division,Team,Name,Wins,Losses,Ties,Percentage,PointsFor,PointsAgainst,NetPoints,Touchdowns,DivisionWins,DivisionLosses,ConferenceWins,ConferenceLosses,TeamID,DivisionTies,ConferenceTies,GlobalTeamID,DivisionRank,ConferenceRank
0,1,2020,AFC,East,BUF,Buffalo Bills,7,3,0,0.700,272,265,7,31,4,0,5,2,4,0,0,4,1,3
1,1,2020,AFC,East,MIA,Miami Dolphins,6,3,0,0.667,251,182,69,27,1,2,3,2,19,0,0,19,2,6
2,1,2020,AFC,East,NE,New England Patriots,4,5,0,0.444,189,211,-22,21,2,1,4,3,21,0,0,21,3,10
3,1,2020,AFC,East,NYJ,New York Jets,0,9,0,0.000,121,268,-147,11,0,4,0,7,24,0,0,24,4,16
4,1,2020,AFC,North,PIT,Pittsburgh Steelers,9,0,0,1.000,271,171,100,33,3,0,6,0,28,0,0,28,1,1


In [22]:
response = requests.get('https://api.sportsdata.io/v3/nfl/scores/json/Schedules/2019REG?key=d8b5ea01537141eb9a320f95994b7109')
schedule_2019 = pd.DataFrame.from_dict(response.json())
schedule_2019.head(25)

,GameKey,SeasonType,Season,Week,Date,AwayTeam,HomeTeam,Channel,PointSpread,OverUnder,StadiumID,Canceled,GeoLat,GeoLong,ForecastTempLow,ForecastTempHigh,ForecastDescription,ForecastWindChill,ForecastWindSpeed,AwayTeamMoneyLine,HomeTeamMoneyLine,Day,DateTime,GlobalGameID,GlobalAwayTeamID,GlobalHomeTeamID,ScoreID,Status,StadiumDetails
0,201910106,1,2019,1,2019-09-05T20:20:00,GB,CHI,NBC,-3.4,53.5,20.0,False,None,None,62.0,67.0,Clear Sky,62.0,7.0,163.0,-199.0,2019-09-05T00:00:00,2019-09-05T20:20:00,16983,12,6,16983,Final,"{'StadiumID': 20, 'Name': 'Soldier Field', 'Ci..."
1,201910105,1,2019,1,2019-09-08T13:00:00,LAR,CAR,FOX,1.9,55.8,26.0,False,None,None,70.0,71.0,Clear Sky,70.0,2.0,-141.0,116.0,2019-09-08T00:00:00,2019-09-08T13:00:00,16989,32,5,16989,Final,"{'StadiumID': 26, 'Name': 'Bank of America Sta..."
2,201910108,1,2019,1,2019-09-08T13:00:00,TEN,CLE,CBS,-6.3,50.1,6.0,False,None,None,59.0,63.0,Overcast Clouds,59.0,9.0,228.0,-285.0,2019-09-08T00:00:00,2019-09-08T13:00:00,16984,34,8,16984,Final,"{'StadiumID': 6, 'Name': 'FirstEnergy Stadium'..."
3,201910115,1,2019,1,2019-09-08T13:00:00,KC,JAX,CBS,4.5,55.8,10.0,False,None,None,77.0,79.0,Clear Sky,77.0,4.0,-233.0,188.0,2019-09-08T00:00:00,2019-09-08T13:00:00,16990,16,15,16990,Final,"{'StadiumID': 10, 'Name': 'TIAA Bank Field', '..."
4,201910119,1,2019,1,2019-09-08T13:00:00,BAL,MIA,CBS,9.1,46.7,2.0,False,None,None,84.0,84.0,Scattered Clouds,84.0,6.0,-359.0,290.0,2019-09-08T00:00:00,2019-09-08T13:00:00,16985,3,19,16985,Final,"{'StadiumID': 2, 'Name': 'Hard Rock Stadium', ..."
5,201910120,1,2019,1,2019-09-08T13:00:00,ATL,MIN,FOX,-4.0,53.5,42.0,False,None,None,57.0,57.0,Light Rain,57.0,9.0,182.0,-222.0,2019-09-08T00:00:00,2019-09-08T13:00:00,16986,2,20,16986,Final,"{'StadiumID': 42, 'Name': 'U.S. Bank Stadium',..."
6,201910124,1,2019,1,2019-09-08T13:00:00,BUF,NYJ,CBS,-2.8,46.7,3.0,False,None,None,55.0,65.0,Broken Clouds,65.0,5.0,142.0,-177.0,2019-09-08T00:00:00,2019-09-08T13:00:00,16987,4,24,16987,Final,"{'StadiumID': 3, 'Name': 'MetLife Stadium', 'C..."
7,201910126,1,2019,1,2019-09-08T13:00:00,WAS,PHI,FOX,-12.0,50.7,18.0,False,None,None,65.0,69.0,Overcast Clouds,65.0,5.0,456.0,-638.0,2019-09-08T00:00:00,2019-09-08T13:00:00,16988,35,26,16988,Final,"{'StadiumID': 18, 'Name': 'Lincoln Financial F..."
8,201910129,1,2019,1,2019-09-08T16:05:00,IND,LAC,CBS,-6.8,51.3,43.0,False,None,None,69.0,70.0,Clear Sky,70.0,1.0,245.0,-313.0,2019-09-08T00:00:00,2019-09-08T16:05:00,16991,14,29,16991,Final,"{'StadiumID': 43, 'Name': 'Dignity Health Spor..."
9,201910130,1,2019,1,2019-09-08T16:05:00,CIN,SEA,CBS,-10.8,50.7,31.0,False,None,None,58.0,59.0,Light Rain,59.0,5.0,399.0,-547.0,2019-09-08T00:00:00,2019-09-08T16:05:00,16992,7,30,16992,Final,"{'StadiumID': 31, 'Name': 'CenturyLink Field',..."


#### From Season Stats I can see us taking Score, Opponent Score, Avg Score and PAssing Dropbacks
#### From Standing we would take Wins, Division Wins, Conference Wins (maybe corresponding losses)


In [10]:
response = requests.get('https://api.sportsdata.io/v3/nfl/scores/json/ScoresByWeek/2020REG/10?key=d8b5ea01537141eb9a320f95994b7109')
current_week_games = pd.DataFrame.from_dict(response.json())
current_week_games = current_week_games[['AwayTeam','HomeTeam']]
current_week_games
for index, row in current_week_games.iterrows():
    print(row.AwayTeam)

IND
HOU
WAS
JAX
PHI
CIN
TB
DEN
NYJ
BUF
SEA
SF
BAL
MIN
LAC


In [20]:

response = requests.get('https://api.sportsdata.io/v3/nfl/scores/json/GameStats/2019REG?key=d8b5ea01537141eb9a320f95994b7109')
team_games_2019 = pd.DataFrame.from_dict(response.json())
team_games_2019.head()


,GameKey,Date,SeasonType,Season,Week,Stadium,PlayingSurface,Temperature,Humidity,WindSpeed,AwayTeam,HomeTeam,AwayScore,HomeScore,TotalScore,OverUnder,PointSpread,AwayScoreQuarter1,AwayScoreQuarter2,AwayScoreQuarter3,AwayScoreQuarter4,AwayScoreOvertime,HomeScoreQuarter1,HomeScoreQuarter2,HomeScoreQuarter3,HomeScoreQuarter4,HomeScoreOvertime,AwayTimeOfPossession,HomeTimeOfPossession,AwayFirstDowns,HomeFirstDowns,AwayFirstDownsByRushing,HomeFirstDownsByRushing,AwayFirstDownsByPassing,HomeFirstDownsByPassing,AwayFirstDownsByPenalty,HomeFirstDownsByPenalty,AwayOffensivePlays,HomeOffensivePlays,AwayOffensiveYards,HomeOffensiveYards,AwayOffensiveYardsPerPlay,HomeOffensiveYardsPerPlay,AwayTouchdowns,HomeTouchdowns,AwayRushingAttempts,HomeRushingAttempts,AwayRushingYards,HomeRushingYards,AwayRushingYardsPerAttempt,HomeRushingYardsPerAttempt,AwayRushingTouchdowns,HomeRushingTouchdowns,AwayPassingAttempts,HomePassingAttempts,AwayPassingCompletions,HomePassingCompletions,AwayPassingYards,HomePassingYards,AwayPassingTouchdowns,HomePassingTouchdowns,AwayPassingInterceptions,HomePassingInterceptions,AwayPassingYardsPerAttempt,HomePassingYardsPerAttempt,AwayPassingYardsPerCompletion,HomePassingYardsPerCompletion,AwayCompletionPercentage,HomeCompletionPercentage,AwayPasserRating,HomePasserRating,AwayThirdDownAttempts,HomeThirdDownAttempts,AwayThirdDownConversions,HomeThirdDownConversions,AwayThirdDownPercentage,HomeThirdDownPercentage,AwayFourthDownAttempts,HomeFourthDownAttempts,AwayFourthDownConversions,HomeFourthDownConversions,AwayFourthDownPercentage,HomeFourthDownPercentage,AwayRedZoneAttempts,HomeRedZoneAttempts,AwayRedZoneConversions,HomeRedZoneConversions,AwayGoalToGoAttempts,HomeGoalToGoAttempts,AwayGoalToGoConversions,HomeGoalToGoConversions,AwayReturnYards,HomeReturnYards,AwayPenalties,HomePenalties,AwayPenaltyYards,HomePenaltyYards,AwayFumbles,HomeFumbles,AwayFumblesLost,HomeFumblesLost,AwayTimesSacked,HomeTimesSacked,AwayTimesSackedYards,HomeTimesSackedYards,AwaySafeties,HomeSafeties,AwayPunts,HomePunts,AwayPuntYards,HomePuntYards,AwayPuntAverage,HomePuntAverage,AwayGiveaways,HomeGiveaways,AwayTakeaways,HomeTakeaways,AwayTurnoverDifferential,HomeTurnoverDifferential,AwayKickoffs,HomeKickoffs,AwayKickoffsInEndZone,HomeKickoffsInEndZone,AwayKickoffTouchbacks,HomeKickoffTouchbacks,AwayPuntsHadBlocked,HomePuntsHadBlocked,AwayPuntNetAverage,HomePuntNetAverage,AwayExtraPointKickingAttempts,HomeExtraPointKickingAttempts,AwayExtraPointKickingConversions,HomeExtraPointKickingConversions,AwayExtraPointsHadBlocked,HomeExtraPointsHadBlocked,AwayExtraPointPassingAttempts,HomeExtraPointPassingAttempts,AwayExtraPointPassingConversions,HomeExtraPointPassingConversions,AwayExtraPointRushingAttempts,HomeExtraPointRushingAttempts,AwayExtraPointRushingConversions,HomeExtraPointRushingConversions,AwayFieldGoalAttempts,HomeFieldGoalAttempts,AwayFieldGoalsMade,HomeFieldGoalsMade,AwayFieldGoalsHadBlocked,HomeFieldGoalsHadBlocked,AwayPuntReturns,HomePuntReturns,AwayPuntReturnYards,HomePuntReturnYards,AwayKickReturns,HomeKickReturns,AwayKickReturnYards,HomeKickReturnYards,AwayInterceptionReturns,HomeInterceptionReturns,AwayInterceptionReturnYards,HomeInterceptionReturnYards,AwaySoloTackles,AwayAssistedTackles,AwayQuarterbackHits,AwayTacklesForLoss,AwaySacks,AwaySackYards,AwayPassesDefended,AwayFumblesForced,AwayFumblesRecovered,AwayFumbleReturnYards,AwayFumbleReturnTouchdowns,AwayInterceptionReturnTouchdowns,AwayBlockedKicks,AwayPuntReturnTouchdowns,AwayPuntReturnLong,AwayKickReturnTouchdowns,AwayKickReturnLong,AwayBlockedKickReturnYards,AwayBlockedKickReturnTouchdowns,AwayFieldGoalReturnYards,AwayFieldGoalReturnTouchdowns,AwayPuntNetYards,HomeSoloTackles,HomeAssistedTackles,HomeQuarterbackHits,HomeTacklesForLoss,HomeSacks,HomeSackYards,HomePassesDefended,HomeFumblesForced,HomeFumblesRecovered,HomeFumbleReturnYards,HomeFumbleReturnTouchdowns,HomeInterceptionReturnTouchdowns,HomeBlockedKicks,HomePuntReturnTouchdowns,HomePuntReturnLong,HomeKickReturnTouchdo

In [17]:
team_games_2019.dtypes

GameKey                              object
Date                                 object
SeasonType                            int64
Season                                int64
Week                                  int64
Stadium                              object
PlayingSurface                       object
Temperature                         float64
Humidity                              int64
WindSpeed                           float64
AwayTeam                             object
HomeTeam                             object
AwayScore                             int64
HomeScore                             int64
TotalScore                            int64
OverUnder                           float64
PointSpread                         float64
AwayScoreQuarter1                     int64
AwayScoreQuarter2                     int64
AwayScoreQuarter3                     int64
AwayScoreQuarter4                     int64
AwayScoreOvertime                     int64
HomeScoreQuarter1               

In [15]:
# create new data frame
teams_season_2019 =  pd.DataFrame()
for index, row in team_games_2019.iterrows():
    # ---------------------------------------update each teams score per week------------------------------------------
    teams_season_2019.loc[row.HomeTeam,row.Week] = row.HomeScore
    teams_season_2019.loc[row.AwayTeam,row.Week] = row.AwayScore
    
    
  
    # ---------------------------------------update each teams games played -------------------------------------------
    if 'GamesPlayed' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'GamesPlayed'] = 0
        teams_season_2019.loc[row.AwayTeam,'GamesPlayed'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['GamesPlayed'] = teams_season_2019['GamesPlayed'].fillna(0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'GamesPlayed'] += 1
    teams_season_2019.loc[row.AwayTeam,'GamesPlayed'] += 1
        

    # ----------------------------------------------update score to week------------------------------------------------
    if 'TotalScore' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalScore'] = 0
        teams_season_2019.loc[row.AwayTeam,'TotalScore'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalScore'] = teams_season_2019['TotalScore'].fillna(0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'TotalScore'] += row.HomeScore 
    teams_season_2019.loc[row.AwayTeam,'TotalScore'] += row.AwayScore
    
    # get avg points scored up to a certain week
    update_avg_home_score = teams_season_2019.loc[row.HomeTeam,'TotalScore'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_avg_away_score = teams_season_2019.loc[row.AwayTeam,'TotalScore'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'avg_up_to_week_{0}'.format(row.Week)] = update_avg_home_score 
    teams_season_2019.loc[row.AwayTeam,'avg_up_to_week_{0}'.format(row.Week)] = update_avg_away_score
    
    teams_season_2019['avg_up_to_week_{0}'.format(row.Week)] = teams_season_2019['avg_up_to_week_{0}'.format(row.Week)].fillna(teams_season_2019['TotalScore'] / (row.Week - 1))
    
    # ----------------------------------- store wins per team up to a certain week ----------------------------------
    if 'TotalWins' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalWins'] = 0
        teams_season_2019.loc[row.AwayTeam,'TotalWins'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalWins'] = teams_season_2019['TotalWins'].fillna(0)
    # check who won
    if row.HomeScore > row.AwayScore:
        teams_season_2019.loc[row.HomeTeam,'TotalWins'] += 1
    else:
        teams_season_2019.loc[row.AwayTeam,'TotalWins'] += 1
    

    teams_season_2019.loc[row.HomeTeam,'wins_up_to_week_{0}'.format(row.Week)] = teams_season_2019.loc[row.HomeTeam,'TotalWins']
    teams_season_2019.loc[row.AwayTeam,'wins_up_to_week_{0}'.format(row.Week)] = teams_season_2019.loc[row.AwayTeam,'TotalWins']
    
    
    # ----------------------------------- store first downs per team up to certain week ----------------------------------

    if 'TotalFirstDowns' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalFirstDowns'] = 0
        teams_season_2019.loc[row.AwayTeam,'TotalFirstDowns'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalFirstDowns'] = teams_season_2019['TotalFirstDowns'].fillna(0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'TotalFirstDowns'] += row.HomeFirstDowns 
    teams_season_2019.loc[row.AwayTeam,'TotalFirstDowns'] += row.AwayFirstDowns
    
    # get avg points scored up to a certain week
    update_first_downs_home = teams_season_2019.loc[row.HomeTeam,'TotalFirstDowns'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_first_downs_away = teams_season_2019.loc[row.AwayTeam,'TotalFirstDowns'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'first_downs_up_to_week_{0}'.format(row.Week)] = update_first_downs_home 
    teams_season_2019.loc[row.AwayTeam,'first_downs_up_to_week_{0}'.format(row.Week)] = update_first_downs_away
    
    teams_season_2019['first_downs_up_to_week_{0}'.format(row.Week)] = teams_season_2019['first_downs_up_to_week_{0}'.format(row.Week)].fillna(teams_season_2019['TotalFirstDowns'] / (row.Week - 1))
    
     # ----------------------------------- calculate third down % per team up to certain week ----------------------------------

    if 'ThirdDownPercentage' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'ThirdDownPercentage'] = 0.0
        teams_season_2019.loc[row.AwayTeam,'ThirdDownPercentage'] = 0.0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['ThirdDownPercentage'] = teams_season_2019['ThirdDownPercentage'].fillna(0.0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'ThirdDownPercentage'] += row.HomeThirdDownPercentage 
    teams_season_2019.loc[row.AwayTeam,'ThirdDownPercentage'] += row.AwayThirdDownPercentage
    
    # get avg points scored up to a certain week
    update_third_down_percentage_home = teams_season_2019.loc[row.HomeTeam,'ThirdDownPercentage'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_third_down_percentage_away = teams_season_2019.loc[row.AwayTeam,'ThirdDownPercentage'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week)] = update_third_down_percentage_home
    teams_season_2019.loc[row.AwayTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week)] = update_third_down_percentage_away
    
    teams_season_2019['third_down_percentage_up_to_week_{0}'.format(row.Week)] = teams_season_2019['third_down_percentage_up_to_week_{0}'.format(row.Week)].fillna(teams_season_2019['ThirdDownPercentage'] / (row.Week - 1))
    
    
teams_season_2019

,1,GamesPlayed,TotalScore,avg_up_to_week_1,TotalWins,wins_up_to_week_1,TotalFirstDowns,first_downs_up_to_week_1,ThirdDownPercentage,third_down_percentage_up_to_week_1,2,avg_up_to_week_2,wins_up_to_week_2,first_downs_up_to_week_2,third_down_percentage_up_to_week_2,3,avg_up_to_week_3,wins_up_to_week_3,first_downs_up_to_week_3,third_down_percentage_up_to_week_3,4,avg_up_to_week_4,wins_up_to_week_4,first_downs_up_to_week_4,third_down_percentage_up_to_week_4,5,avg_up_to_week_5,wins_up_to_week_5,first_downs_up_to_week_5,third_down_percentage_up_to_week_5,6,avg_up_to_week_6,wins_up_to_week_6,first_downs_up_to_week_6,third_down_percentage_up_to_week_6,7,avg_up_to_week_7,wins_up_to_week_7,first_downs_up_to_week_7,third_down_percentage_up_to_week_7,8,avg_up_to_week_8,wins_up_to_week_8,first_downs_up_to_week_8,third_down_percentage_up_to_week_8,9,avg_up_to_week_9,wins_up_to_week_9,first_downs_up_to_week_9,third_down_percentage_up_to_week_9,10,avg_up_to_week_10,wins_up_to_week_10,first_downs_up_to_week_10,third_down_percentage_up_to_week_10,11,avg_up_to_week_11,wins_up_to_week_11,first_downs_up_to_week_11,third_down_percentage_up_to_week_11,12,avg_up_to_week_12,wins_up_to_week_12,first_downs_up_to_week_12,third_down_percentage_up_to_week_12,13,avg_up_to_week_13,wins_up_to_week_13,first_downs_up_to_week_13,third_down_percentage_up_to_week_13,14,avg_up_to_week_14,wins_up_to_week_14,first_downs_up_to_week_14,third_down_percentage_up_to_week_14,15,avg_up_to_week_15,wins_up_to_week_15,first_downs_up_to_week_15,third_down_percentage_up_to_week_15,16,avg_up_to_week_16,wins_up_to_week_16,first_downs_up_to_week_16,third_down_percentage_up_to_week_16,17,avg_up_to_week_17,wins_up_to_week_17,first_downs_up_to_week_17,third_down_percentage_up_to_week_17
CHI,9.0,16.0,494.0,9.0,8.0,0.0,515.0,28.0,981.5,34.7,28.0,18.5,1.0,29.5,41.00,54.0,30.333333,2.0,31.666667,62.866667,28.0,29.750000,3.0,31.000000,60.675000,36.0,31.00,3.0,30.00,61.160,NaN,31.000000,NaN,30.000000,61.160000,43.0,33.000000,3.0,29.666667,55.783333,28.0,32.285714,3.0,31.857143,56.657143,24.0,31.250000,3.0,30.000000,53.912500,35.0,31.666667,4.0,29.222222,51.133333,12.0,29.700000,4.0,29.200000,52.140000,33.0,30.000000,5.0,29.727273,51.609091,42.0,31.000000,6.0,30.416667,54.525000,54.0,32.769231,7.0,31.307692,58.100000,23.0,32.071429,7.0,31.571429,59.521429,9.0,30.533333,7.0,31.533333,60.373333,36.0,30.8750,8.0,32.1875,61.34375
GB,17.0,16.0,653.0,17.0,13.0,1.0,556.0,23.0,1003.1,28.9,36.0,26.5,2.0,29.0,43.30,47.0,33.333333,3.0,28.666667,41.700000,47.0,36.750000,3.0,35.000000,54.875000,59.0,41.20,4.0,36.40,52.560,40.0,41.000000,5.0,36.666667,53.416667,73.0,45.571429,6.0,36.857143,60.642857,54.0,46.625000,7.0,36.750000,66.387500,19.0,43.555556,7.0,35.222222,62.866667,42.0,43.400000,8.0,35.700000,64.270000,NaN,43.400000,NaN,35.700000,64.270000,14.0,40.727273,8.0,35.454545,59.481818,54.0,41.833333,9.0,35.250000,62.291667,35.0,41.307692,10.0,34.923077,62.630769,36.0,40.928571,11.0,33.928571,62.278571,40.0,40.866667,12.0,34.200000,62.253333,40.0,40.8125,13.0,34.7500,62.69375
CAR,47.0,16.0,594.0,47.0,5.0,0.0,580.0,36.0,902.4,78.9,24.0,35.5,0.0,35.5,58.00,66.0,45.666667,1.0,35.666667,61.766667,28.0,41.250000,2.0,35.000000,67.975000,59.0,44.80,3.0,35.60,65.920,64.0,48.000000,4.0,34.500000,60.716667,NaN,48.000000,NaN,34.500000,60.716667,23.0,44.428571,4.0,32.571429,55.857143,52.0,45.375000,5.0,33.250000,56.612500,28.0,43.444444,5.0,34.555556,58.022222,9.0,40.000000,5.0,34.000000,54.700000,54.0,41.272727,5.0,35.000000,55.790909,36.0,40.833333,5.0,35.250000,55.650000,35.0,40.384615,5.0,36.153846,55.369231,42.0,40.500000,5.0,37.071429,57.050000,10.0,38.466667,5.0,36.800000,56.553333,17.0,37.1250,5.0,36.2500,56.40000
LAR,52.0,16.0,683.0,52.0,9.0,1.0,594.0,38.0,1041.6,91.7,47.0,49.5,2.0,36.5,79.20,35.0,44.666667,3.0,36.000000,77.566667,69.0,50.750000,3.0,42.500000,73.950000,50.0,50.60,3.0,42.40,68.620,12.0,44.166667,3.0,38.166667,57.183333,64.0,47.000000,4.0,38.428571,61.385714,42.0,46.375000,5.0,37.750000,65